In [1]:
import os
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from torchsummary import summary
import torch_pruning as tp
from mobilenetv3 import mobilenetv3
from collections import defaultdict
import time
import pandas as pd
from sklearn.model_selection import train_test_split
import pytorch_lightning as L
import h5py
from torch.utils.data import Dataset
import copy

In [2]:
def test_and_plot(loader, model, title_suffix, file_suffix):
    device = next(model.parameters()).device  
    with torch.no_grad():
        predictions_dict = defaultdict(list)
        for batch in loader:
            inputs, true_angles = batch
            inputs, true_angles = inputs.to(device), true_angles.to(device)
            predicted = model(inputs).cpu().numpy()
            true_angles = true_angles.cpu().numpy()
            for i in range(len(true_angles)):
                key = tuple(true_angles[i])
                predictions_dict[key].append(predicted[i])

      
        mean_predictions = {}
        average_base_angles = []

        for key, preds in predictions_dict.items():
            mean_pred = np.mean(preds, axis=0)
            mean_predictions[key] = mean_pred
            average_base_angles.append(mean_pred)

      
        true_azimuths = []
        true_elevations = []
        pred_azimuth_diffs = []
        pred_elevation_diffs = []

        for (true_azimuth, true_elevation), mean_pred in mean_predictions.items():
            true_azimuths.append(true_azimuth)
            true_elevations.append(true_elevation)
            pred_azimuth_diffs.append(mean_pred[0] - true_azimuth)
            pred_elevation_diffs.append(mean_pred[1] - true_elevation)

     
    
        true_azimuths = np.array(true_azimuths)
        true_elevations = np.array(true_elevations)
        pred_azimuth_diffs = np.array(pred_azimuth_diffs)
        pred_elevation_diffs = np.array(pred_elevation_diffs)

        

    return true_azimuths, true_elevations, pred_azimuth_diffs, pred_elevation_diffs


In [3]:
def save_model_size(model_path):
    size_mb = os.path.getsize(model_path) / (1024 * 1024) 
    return size_mb

In [4]:
def save_pruning_groups(pruning_method, groups, prune_amount):
    file_path = f'pruning_groups_{pruning_method}_amount_{prune_amount}.txt'
    with open(file_path, 'w') as f:
        f.write(f"Pruning Method: {pruning_method}\n")
        f.write(f"Pruning Amount: {prune_amount}\n\n")
        for i, group in enumerate(groups):
            f.write(f"Group {i + 1}:\n")
            f.write(group.details())
            f.write("\n" + "-" * 50 + "\n")

In [5]:
class RFClassifier(L.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.lr = 3e-4
        self.lr_ignored = 3e-4  # Learning rate for ignored layers

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        # Initialize lists for different parameter groups
        ignored_layers_params = []
        other_layers_params = []

        # Collect parameters of specific layers based on layer names and conditions
        ignored_params_set = set()
        for name, m in self.model.named_modules():
            if any(name.startswith(f'blocks.{i}') for i in range(3)) or (isinstance(m, torch.nn.Linear) and m.out_features == 2):
                ignored_layers_params += list(m.parameters())
                ignored_params_set.update(m.parameters())  # Add to set to avoid duplicates

        # Other parameters: Exclude the ignored layers' parameters
        other_layers_params = [p for p in self.model.parameters() if p not in ignored_params_set]

        # Create the optimizer with different learning rates for different parameter groups
        optimizer = torch.optim.AdamW([
            {'params': ignored_layers_params, 'lr': self.lr_ignored},  # Lower learning rate for ignored layers
            {'params': other_layers_params, 'lr': self.lr}  # Default learning rate for other layers
        ], weight_decay=0)

        # Learning rate scheduler
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=1)

        return {'optimizer': optimizer, 'lr_scheduler': {'scheduler': lr_scheduler, 'interval': 'step'}}

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('val_loss', loss, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('test_loss', loss, prog_bar=True)

In [6]:
class CustomCheckpoint(L.Callback):
    def __init__(self):
        super().__init__()
        self.best_val_loss = float('inf')
        self.best_model = None

    def on_validation_end(self, trainer, pl_module):
        # Access validation loss from the trainer's metrics
        val_loss = trainer.callback_metrics.get('val_loss')
        if val_loss is not None and val_loss < self.best_val_loss:
            self.best_val_loss = val_loss
            self.best_model = pl_module.model
            # Save the best model
            torch.save(self.best_model, 'best_model.pth')
            print(f"New best model saved with validation loss {val_loss:.4f}")


In [7]:
class R22_H5_Dataset(Dataset):
    def __init__(self, data_file, label='label', iqlabel='iq_data'):
        self.data_file = data_file
        self.label = label
        self.iqlabel = iqlabel

    def __len__(self):
        with h5py.File(self.data_file, 'r') as f:
            length = len(f[self.label])
        return length

    def __getitem__(self, idx):
        with h5py.File(self.data_file, 'r') as f:
            iq_data = f[self.iqlabel][idx]
            label = f[self.label][idx]
        return iq_data, label
    
    def get_metadata(self, idx):
        with h5py.File(self.data_file, 'r') as f:
            metadata = {
                'recording': f['recording'][idx].decode('utf-8)'),
                'category': f['category'][idx].decode('utf-8)')
            }
        return metadata


In [9]:
def prune_model(trained_model, prune_method, prune_amount, train_loader):
    # model.eval()  

    model =  copy.deepcopy(trained_model)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

    example_length = 4096
    example_inputs=torch.randn(1, 8, example_length).to(device)
    # DG = tp.DependencyGraph().build_dependency(model, example_inputs)

    groups = []

    if prune_method == 'channel_pruning_Taylor_importance':
        imp = tp.importance.TaylorImportance() #GroupNormImportance(p=2)

        ignored_layers_group1 = []
        # for m in model.modules():
        #     if isinstance(m, torch.nn.Linear) and m.out_features == 2:
        #         ignored_layers.append(m) # DO NOT prune the final classifier!

        for name, m in model.named_modules():
            # Check if the module is within Sequential(4) or Sequential(5)
            if any(name.startswith(f'blocks.{i}') for i in range(3)) or (isinstance(m, torch.nn.Linear) and m.out_features == 2):
                ignored_layers_group1.append(m)  # DO NOT prune the final classifier

        ignored_layers_group2 = []
        # for m in model.modules():
        #     if isinstance(m, torch.nn.Linear) and m.out_features == 2:
        #         ignored_layers.append(m) # DO NOT prune the final classifier!

        for name, m in model.named_modules():
            # Check if the module is within Sequential(4) or Sequential(5)
            if any(name.startswith(f'blocks.{i+3}') for i in range(3)) or (isinstance(m, torch.nn.Linear) and m.out_features == 2):
                ignored_layers_group2.append(m)  # DO NOT prune the final classifier
    

        # model.to(device)
        # example_inputs.to(device)

        mutual_elements = list(set(ignored_layers_group1) & set(ignored_layers_group2))
        print('mutual elements are:   ', mutual_elements)

        # rf_classifier = RFClassifier(model)#.to(device)
        batch = next(iter(train_loader))
        x, y = batch
        x = x.to(device)
        # x, y = batch
        # x = x.to(device)
       
        iterative_steps= 1

        pruner_group1 = tp.pruner.MagnitudePruner( 
            model,
            example_inputs=x,
            importance=imp,
            pruning_ratio=prune_amount, 
            # pruning_ratio_dict = {model.conv1: 0.2, model.layer2: 0.8}, # customized pruning ratios for layers or blocks
            ignored_layers=ignored_layers_group1,
            iterative_steps= iterative_steps,
            global_pruning=True,
            # isomorphic=True
        )

        pruner_group2 = tp.pruner.MagnitudePruner( 
            model,
            example_inputs=x,
            importance=imp,
            pruning_ratio=0.5, 
            # pruning_ratio_dict = {model.conv1: 0.2, model.layer2: 0.8}, # customized pruning ratios for layers or blocks
            ignored_layers=ignored_layers_group2,
            iterative_steps= iterative_steps,
            global_pruning=True,
            # isomorphic=True
        )

        # prune the model, iteratively if necessary.
        for i in range(iterative_steps):

            # Taylor expansion requires gradients for importance estimation
            if isinstance(imp, tp.importance.TaylorImportance):
                
                x, y = batch
                x = x.to(device)
                y = y.to(device)
                y_hat = model(x)
                loss = F.mse_loss(y_hat, y)
                loss.backward() # before pruner.step()

            pruner_group1.step()
            pruner_group2.step()


  
    save_pruning_groups(prune_method, groups, prune_amount)

   
    input_size = (8, example_length)
    try:
        summary(model, input_size)
        print(f"Model structure matches the input size {input_size}.")
    except RuntimeError as e:
        print(f"Model size mismatch encountered after pruning: {str(e)}")
        print("Adjusting model...")
        summary(model, input_size)
        print("Model is now adjusted and compatible with the input size.")

    return model

In [10]:
original_model_checkpoint = 'august22_beam_t.ckpt'

In [11]:
def train_model_with_pruning(trained_model, pruning_method, model_name_suffix, train_loader, val_loader, prune_amount):
    checkpoint_filename = f"model_pruned_{model_name_suffix}_amount_{prune_amount}.ckpt"
    final_model_path = f'model_final_pruned_{model_name_suffix}_amount_{prune_amount}.pth'

   
    if os.path.exists(checkpoint_filename) or os.path.exists(final_model_path):
        print(f"Model {model_name_suffix} at {prune_amount * 100}% pruning already exists. Skipping training.")
        return


    # model = mobilenetv3(
    #     model_size='mobilenetv3_small_050',
    #     num_classes=2,
    #     drop_rate=0,
    #     drop_path_rate=0,
    #     in_chans=8
    # )

    pruned_model = prune_model(trained_model, pruning_method, prune_amount, train_loader)

    rf_classifier = RFClassifier(pruned_model)
    # rf_classifier.lr = 1e-4

    checkpoint_callback = L.callbacks.ModelCheckpoint(
        dirpath='.',
        filename=checkpoint_filename.replace(".ckpt", ""),
        save_top_k=1,
        verbose=True,
        monitor='val_loss',
        mode='min'
    )
    
    # Create the custom callback
    custom_checkpoint = CustomCheckpoint()

    trainer = L.Trainer(
        max_epochs=100,
        callbacks=[checkpoint_callback, custom_checkpoint],
        accelerator='gpu',
        devices=1,
        benchmark=True,
        precision='32-true',
    )

    print(f"Training the model with {pruning_method} applied at {prune_amount * 100}%...")
    trainer.fit(rf_classifier, train_loader, val_loader)

    # trainer = L.Trainer(
    # accelerator='gpu',
    # callbacks=[checkpoint_callback],
    # devices=1,
    # benchmark=True,
    # precision='bf16-mixed',
    # )

    # print(f"Validating the model with {pruning_method} applied at {prune_amount * 100}%...")
    # trainer.validate(rf_classifier, val_loader)

    torch.save(rf_classifier.model, final_model_path)


    model_size_mb = save_model_size(final_model_path)


    # rf_classifier.eval()
    # start_time = time.time()
    # test_and_plot(
    #     val_loader, rf_classifier.model, f'Pruned {pruning_method} ({prune_amount * 100}% Pruning)',
    #     f'pruned_{pruning_method}_amount_{prune_amount}'
    # )
    # inference_time = time.time() - start_time

    analysis_results['model_name'].append(f"model_pruned_{pruning_method}_amount_{prune_amount}")
    analysis_results['model_size_mb'].append(model_size_mb)
    analysis_results['validation_loss'].append(trainer.callback_metrics['val_loss'].item())
    analysis_results['inference_time'].append(None)
    analysis_results['pruning_percentage'].append(prune_amount * 100)
    analysis_results['pruning_method'].append(pruning_method)

    print(
        f"Model {pruning_method} at {prune_amount * 100}% pruning saved. "
        f"Size: {model_size_mb:.2f} MB, Validation Loss: {trainer.callback_metrics['val_loss'].item()}, "
        # f"Inference Time: {inference_time:.2f} seconds"
    )


In [12]:
train_data = '/home/mohammad.hallaq/workarea/AoA-Pruning/data_h5py_files/r22_train.h5'
val_data = '/home/mohammad.hallaq/workarea/AoA-Pruning/data_h5py_files/r22_test.h5'

In [13]:
example_length = 4096
batch_size = 128
epochs =30

In [14]:
train_set = R22_H5_Dataset(train_data)
val_set = R22_H5_Dataset(val_data)
print(f'Training Set: {len(train_set)} examples')
print(f'Validation Set: {len(val_set)} examples')

num_classes = train_set[0][1].shape[0]

train_loader = torch.utils.data.DataLoader(
    dataset=train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8,
    )
val_loader = torch.utils.data.DataLoader(
    dataset=val_set,
    batch_size=128,
    shuffle=False,
    num_workers=8
    )


Training Set: 298350 examples
Validation Set: 74685 examples


In [15]:
analysis_results = {
    'model_name': [],
    'model_size_mb': [],
    'validation_loss': [],
    'training_time': [],
    'inference_time': [],
    'pruning_percentage': [],
    'pruning_method': [],
}

# print("Testing the original unpruned model...")
model = mobilenetv3(
    model_size='mobilenetv3_small_050',
    num_classes=2,
    drop_rate=0.7,
    drop_path_rate=0.2,
    in_chans=8
)
rf_classifier = RFClassifier.load_from_checkpoint(original_model_checkpoint, model=model)
# rf_classifier.eval()


rf_classifier.to('cuda' if torch.cuda.is_available() else 'cpu')


RFClassifier(
  (model): MobileNetV3(
    (conv_stem): Conv1d(8, 16, kernel_size=(3,), stride=(2,), padding=(1,))
    (bn1): GBN(
      (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop): Identity()
      (act): Hardswish()
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv1d(16, 16, kernel_size=(3,), stride=(2,), padding=(1,), groups=16)
          (bn1): GBN(
            (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
          (aa): Identity()
          (se): SqueezeExcite(
            (conv_reduce): Conv1d(16, 8, kernel_size=(1,), stride=(1,))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv1d(8, 16, kernel_size=(1,), stride=(1,))
          )
          (conv_pw): Conv1d(16, 8, kernel_size=(1,), stride=(1,))
          (bn2): GBN(
  

In [16]:
analysis_results['model_name'].append('original_unpruned')
analysis_results['model_size_mb'].append(save_model_size(original_model_checkpoint))
analysis_results['validation_loss'].append(None)  
analysis_results['training_time'].append(None) 
analysis_results['inference_time'].append(None)
analysis_results['inference_time'].append(None)
analysis_results['pruning_percentage'].append(0)
analysis_results['pruning_method'].append('None')


In [17]:
pruning_methods = ['channel_pruning_Taylor_importance'] #, 'filter_pruning', 'layer_pruning']
pruning_amounts = [0.95]  

In [18]:
for method in pruning_methods:
    for amount in pruning_amounts:
        # print("Skipped a pruning method ...")
        train_model_with_pruning(rf_classifier.model, method, method, train_loader, val_loader, amount)

max_len = max(len(lst) for lst in analysis_results.values())

for key in analysis_results:
    if len(analysis_results[key]) < max_len:
     
        analysis_results[key].extend([None] * (max_len - len(analysis_results[key])))


results_df = pd.DataFrame(analysis_results)
print("This is the analysis result: \n", results_df)

mutual elements are:    [Linear(in_features=1024, out_features=2, bias=True)]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/mohammad.hallaq/workarea/venv/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/mohammad.hallaq/workarea/MobileNet_compression exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/home/mohammad.hallaq/workarea/venv/lib/python3.10/site-packages/torch/_compile.py:24: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  return torch._dynamo.disable(fn, recursive)(*args, **kwargs)

  | Name  | Type        | Params | Mode
---------------------------------------------
0 | model | MobileNetV3 | 20.0 K | eval
---------------------------------------------
20.0 K    Trainable params
0         Non-trainable params
20.0 K    Total params
0.080     Total estimated model para

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 8, 2048]             200
       BatchNorm1d-2              [-1, 8, 2048]              16
         Hardswish-3              [-1, 8, 2048]               0
          Identity-4              [-1, 8, 2048]               0
               GBN-5              [-1, 8, 2048]               0
            Conv1d-6              [-1, 8, 1024]              32
       BatchNorm1d-7              [-1, 8, 1024]              16
              ReLU-8              [-1, 8, 1024]               0
          Identity-9              [-1, 8, 1024]               0
              GBN-10              [-1, 8, 1024]               0
         Identity-11              [-1, 8, 1024]               0
           Conv1d-12                 [-1, 3, 1]              27
             SiLU-13                 [-1, 3, 1]               0
           Conv1d-14                 [-

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

New best model saved with validation loss 1441.6115


...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 98, global step 230769: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 99, global step 233100: 'val_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=100` reached.


Model channel_pruning_Taylor_importance at 95.0% pruning saved. Size: 0.24 MB, Validation Loss: 11.844345092773438, 
This is the analysis result: 
                                           model_name  model_size_mb  \
0                                  original_unpruned       6.526157   
1  model_pruned_channel_pruning_Taylor_importance...       0.240189   
2                                               None            NaN   

   validation_loss training_time inference_time  pruning_percentage  \
0              NaN          None           None                 0.0   
1        11.844345          None           None                95.0   
2              NaN          None           None                 NaN   

                      pruning_method  
0                               None  
1  channel_pruning_Taylor_importance  
2                               None  
